In [48]:
#from telegram import Update, Bot
import json
import time
import pandas as pd
from datetime import datetime, date, timedelta
from pybit import usdt_perpetual
import numpy as np
#from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes
#-1001839931719
from dotenv import dotenv_values
#import nest_asyncio

class dataAssembler:

    long_alert = 30
    short_alert = 70
    last_tsi = 0
    bot = None
    eth_data = pd.DataFrame(data=None, columns=["open_time", "high", "low","open","close","volume","turnover", "date_time"])
        

    def get_tsi_and_signal(self, close, long, short, signal):
        diff = close - close.shift(1)
        abs_diff = abs(diff)
        
        diff_smoothed = diff.ewm(span = long, adjust = False).mean()
        diff_double_smoothed = diff_smoothed.ewm(span = short, adjust = False).mean()
        abs_diff_smoothed = abs_diff.ewm(span = long, adjust = False).mean()
        abs_diff_double_smoothed = abs_diff_smoothed.ewm(span = short, adjust = False).mean()
        
        tsi = (diff_double_smoothed / abs_diff_double_smoothed) * 100
        signal = tsi.ewm(span = signal, adjust = False).mean()
        #tsi = tsi[tsi.index >= '2020-01-01'].dropna()
        #signal = signal[signal.index >= '2020-01-01'].dropna()
        
        return tsi, signal

    def get_tsi_and_signal_NORMALIZED(self, close, long, short, signal):
        diff = close - close.shift(1)
        abs_diff = abs(diff)
        
        diff_smoothed = diff.ewm(span = long, adjust = False).mean()
        diff_double_smoothed = diff_smoothed.ewm(span = short, adjust = False).mean()
        abs_diff_smoothed = abs_diff.ewm(span = long, adjust = False).mean()
        abs_diff_double_smoothed = abs_diff_smoothed.ewm(span = short, adjust = False).mean()
        
        tsi = (diff_double_smoothed / abs_diff_double_smoothed) * 100
        signal = tsi.ewm(span = signal, adjust = False).mean()

        TSI100 = pd.Series(np.zeros(len(tsi)))

        for i in range(1, len(tsi)):
            TSI100 = 100-100/(1+tsi[i])
        print(TSI100.tail(), tsi.tail())


        #tsi = tsi[tsi.index >= '2020-01-01'].dropna()
        #signal = signal[signal.index >= '2020-01-01'].dropna()
        
        return tsi, signal

    def assemble_data(self, symbol, intervals, minutesInterval, tsiPeriod1=25, tsiPeriod2=13, tsiPeriod3=12):
        print('assembling')
        #example: (10,ETHUSDT,200,1) for 10*200 of 1min.
        eth_data = self.get_minutes(intervals, symbol, 200, minutesInterval)
        eth_data['tsi'], eth_data['signal_line'] = self.get_tsi_and_signal(eth_data['close'], tsiPeriod1, tsiPeriod2, tsiPeriod3)
        self.eth_data = eth_data
        self.last_tsi = eth_data['tsi'][::-1]
        #eth = eth[eth['date_time'] >= '2020-01-01']
        eth_data.tail(1)
        return eth_data

    async def get_last_tsi(self):
        updatedPrice = self.get_minutes(1, "ETHUSDT", 1, 1)
        new_array = pd.concat([self.eth_data, updatedPrice])
        new_array['tsi'], new_array['signal_line'] = self.get_tsi_and_signal(new_array['close'], 25, 13, 12)
        self.last_tsi = new_array['tsi'][-1]
        self.eth_data = new_array

        await self.bot.send_message(chat_id, f'new tsi at {pd.Timestamp(datetime.now())}: {self.last_tsi}')


    def get_minutes(iterations, symbol, number_of_records, minutes_interval):
        result = []
        session_unauth = usdt_perpetual.HTTP(
            endpoint="https://api.bybit.com"
        )
        j = 0
        for i in range(iterations, 0, -1):
            _time = int(pd.Timestamp(datetime.now()-timedelta(minutes=(minutes_interval*number_of_records*i)), tz='US/Pacific').timestamp())
            res = session_unauth.query_kline(
                symbol=symbol,
                interval=minutes_interval,
                from_time=_time
            )["result"]
            j += 1
            print(f'processed {j} of {iterations}')
            result += res

        df = pd.DataFrame(data=result, columns=["open_time", "high", "low","open","close","volume","turnover", "date_time"])

        df['date_time'] = df['open_time'].apply(lambda x: datetime.fromtimestamp(x))
        df.set_index('date_time').astype(float)
        return df

    def rsi(close, rsilen):
        f=0
        d=0
        rsi_len = rsilen
        change_close = pd.Series(np.zeros(l))
        gain = pd.Series(np.zeros(l))
        loss = pd.Series(np.zeros(l))
        smagain = pd.Series(np.zeros(l))
        smaloss = pd.Series(np.zeros(l))
        RS = pd.Series(np.zeros(l))
        RSI100 = pd.Series(np.zeros(l))
        dRdt = pd.Series(np.zeros(l))
        dR2dt = pd.Series(np.zeros(l))
        for i in range(1,len(close)):   
            #print(close[i])
            change_close[i] = close[i]-close[i-1]
            if change_close[i] > 0:
                gain[i] = change_close[i]
            elif change_close[i] < 0:
                loss[i] = change_close[i]*-1
            elif change_close[i] == 0.00000:
                loss[i] = 0
                gain[i] = 0
            if i == rsi_len:   
                smagain[rsi_len] = gain[i-rsi_len:i].sum()/rsi_len
                smaloss[rsi_len] = loss[i-rsi_len:i].sum()/rsi_len
            elif i > rsi_len: 
                smagain[i] = gain[i-rsi_len:i].sum()/rsi_len
                smaloss[i] = loss[i-rsi_len:i].sum()/rsi_len
                if smaloss[i] == 0:
                    RSI100[i] = 100
                else:
                    RS[i] = smagain[i]/smaloss[i]
                    RSI100[i] = 100 - 100/(1+RS[i])
                #print(RSIclose[i])
                
        return RSI100

    def write_df_to_json(df, filename="data.json"):
        dataJson = df.to_json(orient='records')
        with open(filename, "w") as i :
            json.dump(dataJson, i)
    
    def write_to_json(_json, filename="data.json"):
        with open(filename, "w") as i :
            json.dump(_json, i)

    def load_json(filename='data.json'):
        # Opening JSON file
        f = open(filename,)
        # returns JSON object as 
        # a dictionary
        loaded_json = json.load(f)
        #pd_json = pd.read_json(loaded_json, orient='Columns')
        f.close()
        return loaded_json

    def load_df(filename='data.json'):
        # Opening JSON file
        f = open(f"./json/{filename}",)
        # returns JSON object as 
        # a dictionary
        loaded_json = json.load(f)
        pd_json = pd.read_json(loaded_json, orient='Columns')
        f.close()
        return pd_json
        


In [ ]:
#get_minutes(iterations, symbol, number_of_records, minutes_interval=1)
data_1m = dataAssembler.get_minutes(100,"BTCUSDT", 200, 1)
data_5m = dataAssembler.get_minutes(100,"BTCUSDT", 200, 5)

dataAssembler.write_df_to_json(data_1m, 'data_1m_100.json')
dataAssembler.write_df_to_json(data_5m, 'data_5m_100.json')



In [49]:
data_1m_100 = dataAssembler.load_df('data_1m_100.json')

In [50]:
data_1m_100.tail()

,open_time,high,low,open,close,volume,turnover,date_time
19995,2023-01-25 19:08:00,22709.5,22690.5,22691.0,22705.5,35.129,7.974808e+05,2023-01-25 11:08:00
19996,2023-01-25 19:09:00,22705.5,22692.5,22705.5,22692.5,46.890,1.064286e+06,2023-01-25 11:09:00
19997,2023-01-25 19:10:00,22704.0,22692.5,22692.5,22704.0,17.326,3.932476e+05,2023-01-25 11:10:00
19998,2023-01-25 19:11:00,22711.0,22702.5,22704.0,22711.0,39.429,8.952330e+05,2023-01-25 11:11:00
19999,2023-01-25 19:12:00,22711.0,22710.5,22711.0,22711.0,2.569,5.834453e+04,2023-01-25 11:12:00


In [ ]:
#dataAssembler.write_to_json(dataJson)

In [ ]:
# dataFromJson = pd.DataFrame.from_dict(pd.json_normalize(dataJson))
# dataFromJson

In [ ]:
# pd.read_json('./data.json', orient='records')

In [ ]:
# # Opening JSON file
# f = open('data.json',)
# # returns JSON object as 
# # a dictionary
# loaded_json = json.load(f)
# pd_json = pd.read_json(loaded_json, orient='Columns')

In [ ]:
#pd_json = pd.read_json(loaded_json, orient='Columns')

In [ ]:
#pd_json